<a href="https://colab.research.google.com/github/cassions/FIIsCrawler/blob/main/fii.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests as req
import re
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np


In [ ]:
urlLista='https://www.fundamentus.com.br/fii_resultado.php'

headers = {
    'User-Agent'      : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    'Accept'          : 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language' : 'en-US,en;q=0.5',
    'DNT'             : '1',
    'Connection'      : 'close'
}

htmlLista = req.get(urlLista,headers=headers)

In [ ]:
soupLista = bs(htmlLista.content, 'html.parser')
codesHtml = soupLista.find('tbody').find_all('a')
codes=[]
for code in codesHtml:
  codes.append(code.text.lower())

https://investidor10.com.br/fiis/zifi11


In [ ]:
url='https://investidor10.com.br/fiis/'

def getFii(code,dfFiis,url,debug=0):
  url=url+code
  if debug == 1:
    print(url)
  htmlFii = req.get(url,headers=headers)
  if htmlFii.status_code == 200:
    soupFii = bs(htmlFii.content, 'html.parser')
    # Pega dados da tabela no meio da pagina
    cells = soupFii.find('div',{'id':'table-indicators'}).find_all('span')
    i=0
    infos = {}
    for cell in cells:
      if i%2 == 0:
        chave = cell.text.strip()
      if i%2 == 1:
        infos[chave]=cell.text.strip()
      i=i+1

    segmento = infos['SEGMENTO']
    numCotistas = infos['NUMERO DE COTISTAS']
    vacancia = infos['VACÂNCIA']
    tipo = infos['TIPO DE FUNDO']

    # Pega dados do topo da pagina
    cotacao = re.compile(r'\d+\,\d*').search(soupFii.find('div',{'class':'_card cotacao'}).text)
    if cotacao:
      cotacao = cotacao.group()
    dy12m = re.compile(r'\d+\,\d*').search(soupFii.find('div',{'class':'_card dy'}).text)
    if dy12m:
      dy12m = dy12m.group()
    pvp = re.compile(r'\d+\,\d*').search(soupFii.find('div',{'class':'_card vp'}).text)
    if pvp:
      pvp = pvp.group()

    row = pd.DataFrame({"code":[code],"segmento":[segmento],"numCotistas":[numCotistas],"vacancia":[vacancia],"tipo":[tipo],"cotacao":[cotacao],"dy12m":[dy12m],"pvp":[pvp]})

    dfFiis = pd.concat([dfFiis, row], ignore_index=True)

  return dfFiis

In [ ]:
dfFiis = pd.DataFrame({"code":[],"segmento":[],"numCotistas":[],"vacancia":[],"tipo":[],"cotacao":[],"dy12m":[],"pvp":[]})
for code in codes:
  dfFiis = getFii(code,dfFiis,url,1)

https://investidor10.com.br/fiis/aazq11
https://investidor10.com.br/fiis/abcp11
https://investidor10.com.br/fiis/aefi11
https://investidor10.com.br/fiis/afcr11
https://investidor10.com.br/fiis/afhi11
https://investidor10.com.br/fiis/afof11
https://investidor10.com.br/fiis/agcx11
https://investidor10.com.br/fiis/agrx11
https://investidor10.com.br/fiis/aiec11
https://investidor10.com.br/fiis/ajfi11
https://investidor10.com.br/fiis/almi11
https://investidor10.com.br/fiis/alzc11
https://investidor10.com.br/fiis/alzm11
https://investidor10.com.br/fiis/alzr11
https://investidor10.com.br/fiis/ancr11
https://investidor10.com.br/fiis/apto11
https://investidor10.com.br/fiis/apxm11
https://investidor10.com.br/fiis/arct11
https://investidor10.com.br/fiis/aroa11
https://investidor10.com.br/fiis/arri11
https://investidor10.com.br/fiis/arxd11
https://investidor10.com.br/fiis/asmt11
https://investidor10.com.br/fiis/asrf11
https://investidor10.com.br/fiis/atsa11
https://investidor10.com.br/fiis/aurb11


In [ ]:
dfFiis.dtypes

code            object
segmento        object
numCotistas    float64
vacancia       float64
tipo            object
cotacao        float64
dy12m          float64
pvp            float64
dtype: object

In [ ]:
dfFiisBkp = dfFiis.copy()

In [ ]:
dfFiis = dfFiisBkp.copy()

In [ ]:
dfFiis['numCotistas'] = dfFiis['numCotistas'].str.replace('-', '').str.replace('.', '')
dfFiis['numCotistas'] = pd.to_numeric(dfFiis['numCotistas'], errors='coerce',downcast="integer")

dfFiis['cotacao'] = dfFiis['cotacao'].str.replace(',', '.')
dfFiis['cotacao'] = pd.to_numeric(dfFiis['cotacao'], errors='coerce')

dfFiis['dy12m'] = dfFiis['dy12m'].str.replace(',', '.')
dfFiis['dy12m'] = pd.to_numeric(dfFiis['dy12m'], errors='coerce')

dfFiis['pvp'] = dfFiis['pvp'].str.replace(',', '.')
dfFiis['pvp'] = pd.to_numeric(dfFiis['pvp'], errors='coerce')

dfFiis['vacancia'] = dfFiis['vacancia'].str.replace(',', '.').str.replace('%', '')
dfFiis['vacancia'] = pd.to_numeric(dfFiis['vacancia'], errors='coerce')
dfFiis['vacancia'] = dfFiis['vacancia']/100

In [ ]:

dfFiis.to_excel('/content/drive/MyDrive/Personal/Financeiro_Contab/Investimentos/fiis.xlsx', index=False, sheet_name='Fiis')


In [ ]:
dfFiis = pd.read_excel('/content/drive/MyDrive/Personal/Financeiro_Contab/Investimentos/fiis.xlsx', sheet_name='Fiis')

In [ ]:
dfFiis['segmento'].unique()

array(['Fiagros', 'Shoppings / Varejo', 'Outros', 'Híbrido',
       'Títulos e Valores Mobiliários', 'Lajes Corporativas',
       'Logístico / Indústria / Galpões', 'Agências bancárias', 'Hotéis',
       'Fundo de Fundos', 'Educacional', 'Hospitalar'], dtype=object)

In [ ]:
excludedSegs = ['Hotéis','Hospitalar']
dfFiis[
    (~dfFiis['segmento'].isin(excludedSegs)) &
    (~dfFiis['tipo'].isin(['Fundo de papel'])) &
    (dfFiis['dy12m'] > 9) &
    (dfFiis['pvp'] < 1) &
    (dfFiis['vacancia'].notnull())&
    (dfFiis['vacancia'] < 0.1)
    ].sort_values(by=['numCotistas'], ascending=False)

,code,segmento,numCotistas,vacancia,tipo,cotacao,dy12m,pvp
394,visc11,Shoppings / Varejo,355184.0,0.0620,Fundo de Tijolo,113.82,10.14,0.90
172,hsml11,Shoppings / Varejo,195668.0,0.0500,Fundo de Tijolo,97.05,9.50,0.89
393,vino11,Lajes Corporativas,153317.0,0.0300,Fundo de Tijolo,6.40,11.12,0.60
229,mall11,Shoppings / Varejo,137655.0,0.0600,Fundo de Tijolo,113.57,9.05,0.95
228,lvbi11,Logístico / Indústria / Galpões,127139.0,0.0080,Fundo de Tijolo,115.50,9.13,0.94
48,brco11,Logístico / Indústria / Galpões,123094.0,0.0600,Fundo de Tijolo,117.86,9.02,0.99
154,hgbs11,Shoppings / Varejo,122318.0,0.0670,Fundo de Tijolo,216.74,10.18,0.96
395,viur11,Outros,51916.0,0.0300,Fundo de Tijolo,6.69,12.54,0.76
171,hslg11,Logístico / Indústria / Galpões,38480.0,0.0480,Fundo de Tijolo,87.77,10.09,0.84
270,patl11,Logístico / Indústria / Galpões,25807.0,0.0270,Fundo de Tijolo,61.33,11.64,0.62
